In [1]:
import sys
import operator
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import re
import unicodedata
from operator import add
from itertools import combinations

In [2]:
spark = SparkSession \
            .builder \
            .appName("clustering") \
            .getOrCreate()
sc = spark.sparkContext

In [14]:
data = spark.read.csv('inspections.csv', header = True)

In [19]:
data2 = data.rdd.map(lambda x:(x['facility_name'], x['facility_address']))\
.distinct().map(lambda x:(x[0], x[0]+ ' '+x[1])).cache()

In [20]:
data2.take(10)

[('7 ELEVEN #37215A', '7 ELEVEN #37215A 3330 W FLORENCE AVE'),
 ('ABC DONUTS', 'ABC DONUTS 3027 N SAN FERNANDO RD UNIT 103'),
 ('EL POLLO LOCO', 'EL POLLO LOCO 5319 W SUNSET BLVD'),
 ('FOOD COURT TRAY WASHING AREA',
  'FOOD COURT TRAY WASHING AREA 928 S WESTERN AVE STE 300'),
 ('ORANGE TREE CAFE', 'ORANGE TREE CAFE 3500 W 6TH ST STE #220'),
 ("NORM'S RESTAURANT", "NORM'S RESTAURANT 470 N LA CIENEGA BLVD"),
 ('BEVERLY FALAFEL', 'BEVERLY FALAFEL 8508 W 3RD ST'),
 ('KRISPY KRUNCHY CHICKEN - ALVARADO',
  'KRISPY KRUNCHY CHICKEN - ALVARADO 600 S ALVARADO ST STE 111C'),
 ('CAFE 153', 'CAFE 153 1125 3/4 WALL ST'),
 ('DUNES RESTAURANT', 'DUNES RESTAURANT 5625 W SUNSET BLVD')]

In [21]:
names = data2.map(lambda x: (x[0], x[1].strip()))\
                .map(lambda x: (x[0], x[1].lower()))\
                .map(lambda x: (x[0], re.sub(r'[^\w\s]','', x[1])))\
                .map(lambda x: (x[0], unicodedata.normalize('NFC',x[1])))\
                .map(lambda x: (x[0], x[1].split(' ')))\
                .map(lambda x: (x[0], list(filter(None, x[1])))).cache()

In [22]:
print(names.take(10))

[('7 ELEVEN #37215A', ['7', 'eleven', '37215a', '3330', 'w', 'florence', 'ave']), ('ABC DONUTS', ['abc', 'donuts', '3027', 'n', 'san', 'fernando', 'rd', 'unit', '103']), ('EL POLLO LOCO', ['el', 'pollo', 'loco', '5319', 'w', 'sunset', 'blvd']), ('FOOD COURT TRAY WASHING AREA', ['food', 'court', 'tray', 'washing', 'area', '928', 's', 'western', 'ave', 'ste', '300']), ('ORANGE TREE CAFE', ['orange', 'tree', 'cafe', '3500', 'w', '6th', 'st', 'ste', '220']), ("NORM'S RESTAURANT", ['norms', 'restaurant', '470', 'n', 'la', 'cienega', 'blvd']), ('BEVERLY FALAFEL', ['beverly', 'falafel', '8508', 'w', '3rd', 'st']), ('KRISPY KRUNCHY CHICKEN - ALVARADO', ['krispy', 'krunchy', 'chicken', 'alvarado', '600', 's', 'alvarado', 'st', 'ste', '111c']), ('CAFE 153', ['cafe', '153', '1125', '34', 'wall', 'st']), ('DUNES RESTAURANT', ['dunes', 'restaurant', '5625', 'w', 'sunset', 'blvd'])]


In [23]:
all_names=names.collect()
name_dict ={}
for each in all_names:
    name_dict[each[0]]=each[1]

In [24]:
tokens = names.map(lambda x: x[1]).collect()

In [25]:
shingles = []
for each in tokens:
    shingles.extend(each)
shingles = sorted(list(set(shingles)))
rep_shingles = {}
i = 1
for each in shingles:
    rep_shingles[each] = i
    i = i+1

In [26]:
print(len(shingles))

14542


In [23]:
print(rep_shingles)

{'001': 1, '0026': 2, '0027': 3, '003': 4, '004554': 5, '007': 6, '0078': 7, '013': 8, '016': 9, '026176': 10, '033': 11, '0375': 12, '0409': 13, '041': 14, '0410': 15, '041500': 16, '051': 17, '05423': 18, '05425': 19, '05426': 20, '05430': 21, '05434': 22, '05435': 23, '05436': 24, '05456': 25, '05457': 26, '05458': 27, '05879': 28, '059': 29, '06023': 30, '06211': 31, '063': 32, '06516': 33, '06854': 34, '06931': 35, '06976': 36, '07016': 37, '07036': 38, '0717': 39, '07482': 40, '083': 41, '0876': 42, '09136': 43, '09144': 44, '09285': 45, '09563': 46, '09985': 47, '1': 48, '10': 49, '100': 50, '1000': 51, '1003': 52, '10070': 53, '101': 54, '10107': 55, '10129': 56, '102': 57, '1021': 58, '10291': 59, '10292': 60, '10295': 61, '10296': 62, '103': 63, '10309': 64, '1039': 65, '103rd': 66, '104': 67, '10433': 68, '10469': 69, '105': 70, '10509': 71, '10586': 72, '105meat': 73, '106': 74, '1061': 75, '10614': 76, '1064': 77, '10663': 78, '107': 79, '10724': 80, '10734': 81, '10791': 

In [27]:
trans_names = names.mapValues(lambda x: [rep_shingles[i] for i in x]).cache()

In [28]:
print(trans_names.take(20))

[('7 ELEVEN #37215A', [4936, 8474, 2984, 2692, 14165, 8816, 6447]), ('ABC DONUTS', [6006, 8293, 2458, 11343, 12771, 8750, 12387, 13982, 148]), ('EL POLLO LOCO', [8463, 12131, 10579, 4054, 14165, 13401, 6903]), ('FOOD COURT TRAY WASHING AREA', [8843, 7904, 13864, 14208, 6350, 5776, 12679, 14250, 6447, 13297, 2428]), ('ORANGE TREE CAFE', [11679, 13869, 7208, 2827, 14165, 4935, 13269, 13297, 1838]), ("NORM'S RESTAURANT", [11522, 12454, 3659, 11343, 10339, 7644, 6903]), ('BEVERLY FALAFEL', [6791, 8667, 5508, 14165, 3193, 13269]), ('KRISPY KRUNCHY CHICKEN - ALVARADO', [10304, 10308, 7532, 6189, 4487, 12679, 6189, 13269, 13297, 474]), ('CAFE 153', [7208, 1250, 496, 2739, 14193, 13269]), ('DUNES RESTAURANT', [8358, 12454, 4263, 14165, 13401, 6903]), ('ONO HAWAIIAN BBQ', [11666, 9472, 6672, 4204, 14165, 13401, 6903]), ('MUSH BAKERY', [11325, 6545, 3995, 14165, 13401, 6903]), ('HOT THAI RESTAURANT', [9641, 13680, 12454, 3320, 12679, 12771, 14089, 6903]), ('ALEX FISH MARKET', [6141, 8786, 10870,

In [29]:
def min_hash(values):
    sign = []
    for i in range(1, 101):
        hash_list = []
        for v in values:
            h_value = (2*v + 11*i) % 100
            hash_list.append(h_value)
        sign.append(min(hash_list))
    return sign
def band(item):
    band = []
    for i in range(0, 2):
        band.append((i, (item[0], item[1][i])))
    return band



In [30]:
def cluster(values):
    Output = {} 
    for x, y in values: 
        if tuple(y) in Output: 
            Output[tuple(y)].append((x)) 
        else: 
            Output[tuple(y)] = [(x)]
    return Output

In [31]:
def clean(l):
    output = []
    for each in l:
        if len(each[1])>=2:
            output.extend(each(1))
    return output

In [32]:
signiture = trans_names.mapValues(lambda x: min_hash(x))\
                        .mapValues(lambda x: [x[i:i+50] for i in range(0, 100, 50)])\
                        .flatMap(lambda x: band(x))\
                        .groupByKey()\
                        .map(lambda x: (x[0],list(x[1])))\
                        .mapValues(lambda x: cluster(x))\
                        .map(lambda x: list(x[1].items())).cache()

In [33]:
candidates = signiture.collect()
cand = []
for each in candidates:
    cand.extend(each)
candidate = []
for each in cand:
    if len(each[1])>=2:
        candidate.append(each[1])
print(candidate)

[['BLD', 'BLD'], ['JAMBA JUICE #527', 'JAMBA JUICE #527'], ['CARLS JR #321', "CARL'S JR #321"], ['JACK IN THE BOX # 3215', 'JACK IN THE BOX #3215'], ['LITTLE CAESAR #5909', 'HILARY MINI MARKET'], ['RAHEL VEGAN CUISINE', 'RAHEL VEGGIE CUISINE'], ['ALEX SANDWICH', 'ALEX SANDWICH SHOP'], ['POKE BAR', 'CVS/PHARMACY #8892', 'TROPICAL CLUB'], ['OLD DVIN', 'CAFFE VITA'], ['POLLO CAMPERO', 'POLLO CAMPERO #24'], ['YOGURTLAND', 'J MARKET', 'PINKBERRY'], ['CALIFORNIA RAMEN FACTORY', 'LOS TAPATIOS RESTAURANT'], ['CONCHITAS RESTAURANT', 'WING FACTORY'], ['WHOLE FOODS MARKET', 'KIKKA @ WHOLE FOODS MARKET'], ['CASEY S BAR AND GRILL', 'OCHO MEXICAN GRILL'], ['BOBALICIOUS', 'TARGET'], ["DENNY'S", 'SLAUSON SHELL'], ['HIGHLAND MARKET & DELI', 'HIGHLAND DELI / MARKET'], ['TEXIS RESTAURANT', 'SMART & FINAL'], ['ASAKUMA RICE', 'SPRINKLES CUPCAKES'], ['PANDA KING', 'EL FUERTE MARKET'], ['ONO HAWAIIAN BBQ', 'MAPAYA'], ['BIRRIERIA JALISCO', 'BOSCO CAFE BAKERY'], ['G-STATION', 'SPL. COFFEE'], ["RODEO'S GRILL", 

In [34]:
print(len(candidate))

537


In [35]:
def similarity(values):
    result = []
    permutations = list(combinations(values, 2))
    for each in permutations:
        u1 = each[0]
        u2 = each[1]
        m1 = name_dict[u1]
        m2 = name_dict[u2]
        inter = set(m1).intersection(set(m2))
        union = set(m1).union(set(m1))
        simi = float(len(inter)/len(union))
        if simi>=0.9:
            result.append(each)
    return result

In [36]:
sc_cand = sc.parallelize(candidate).map(lambda x: similarity(x))

In [43]:
results = sc_cand.collect()
result = []
for each in results:
    if len(each)>0:
        result.append(each)
output = []
for i in result:
    clusters = []
    for each in i:
        clusters.append(each[0])
        clusters.append(each[1])
#     clusters = list(set(clusters))
    output.append(clusters)
output2 = list(set([tuple(t) for t in output]))

In [44]:
output2

[('23RD ST CAFE', '23RD ST. CAFE'),
 ('TEN YO ENTERPRISE INC', 'TEN YO ENTERPRISE INC'),
 ('L & G MINI MARKET DISCOUNT', 'L & G MINI MARKET'),
 ('E & M Market', 'E & M MEAT MARKET'),
 ('EL GALLITO RESTAURANTE', 'EL GALLITO RESTAURANTE'),
 ('HAN SHIK', 'HAN SHIK'),
 ('INTI PERUVIAN RESTAURANT', 'INTI PERUVIAN RESTAURANT'),
 ("DOMINO'S PIZZA", "DOMINO'S PIZZA"),
 ('CARLS JR #321', "CARL'S JR #321"),
 ('VILLAMAR MARKET', 'VILLAMAR MARKET'),
 ('QUE DELICIOUS', 'QUE DELICIOUS'),
 ('CHINA BOWL', 'CHINA BOWL EXPRESS'),
 ('LOUISIANA', 'LOUISIANA FRIED CHICKEN'),
 ('NIZAM INDIAN CUISINE', 'NIZAM INDIAN CUISINE'),
 ('CRENSHAW DISCOUNT STORE', 'CRENSHAW DISCOUNT STORE'),
 ('HARVEST CAFE', 'HARVEST CAFE'),
 ('WHEALTHY RESTAURANT', 'WHEALTHY RESTAURANT'),
 ('OLYMPIC BURGER', 'OLYMPIC BURGER'),
 ('CHINA EXPRESS', 'CHINA EXPRESS'),
 ("PACO'S LIQUOR", 'PACOS LIQUOR'),
 ('LOS MORELIANOS', 'LOS MORELIANOS'),
 ('MARLIN RESTAURANT', 'MARLIN RESTAURANT'),
 ('MANDARIN HOUSE', 'MANDARIN HOUSE'),
 ('EDIBLE AR

In [46]:
print(len(output2))

109


In [47]:
import csv
with open('la_mat_lsh.csv', "w") as f:
    w = csv.writer(f)
    w.writerows(output2)